# PySpark Interactive Example Code

To run this notebook on Midway 3, follow the instructions in `spark-sinteractive-setup.md`, included in this directory.

Once you have followed the instructions to launch a Jupyter Lab server on Midway 3, import relevant PySpark functions and start the Spark session like so:

In [1]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

# Start Spark Session
spark = SparkSession.builder.getOrCreate()

Read Amazon book review data for our shared directory:

In [2]:
data = spark.read.csv('/project/macs30123/AWS_book_reviews/*.csv',
                      header='true',
                      inferSchema='true')

data.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- star_rating: string (nullable = true)
 |-- helpful_votes: string (nullable = true)
 |-- total_votes: string (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)
 |-- year: string (nullable = true)



Recast the columns we'll be working with to the correct data type:

In [3]:
data = data.withColumn('star_rating', F.col('star_rating').cast('int')) \
           .withColumn('total_votes', F.col('total_votes').cast('int')) \
           .withColumn('helpful_votes', F.col('helpful_votes').cast('int'))

data.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)
 |-- year: string (nullable = true)



Summarize data by star_rating:

In [4]:
stars_votes = data.groupBy('star_rating') \
                  .sum('total_votes', 'helpful_votes') \
                  .sort('star_rating', ascending=False)

stars_votes.show()

+-----------+----------------+------------------+
|star_rating|sum(total_votes)|sum(helpful_votes)|
+-----------+----------------+------------------+
|       1984|               5|              null|
|       1956|              56|              null|
|       1861|            null|              null|
|       1632|               6|              null|
|         30|              41|              null|
|         10|               5|              null|
|          5|        54772802|          44796242|
|          4|        13946766|          11094517|
|          3|        10113149|           7019293|
|          2|         9006573|           5579123|
|          1|        22610642|          10978974|
|          0|              81|              null|
|       null|           44630|             38760|
+-----------+----------------+------------------+



Drop rows with null values and then print out resulting data:

In [5]:
stars_votes_clean = stars_votes.dropna()
stars_votes_clean.show()

+-----------+----------------+------------------+
|star_rating|sum(total_votes)|sum(helpful_votes)|
+-----------+----------------+------------------+
|          5|        54772802|          44796242|
|          4|        13946766|          11094517|
|          3|        10113149|           7019293|
|          2|         9006573|           5579123|
|          1|        22610642|          10978974|
+-----------+----------------+------------------+

